# Naive Bayes

##### Bayes’s Theorem

According to the Wikipedia, In probability theory and statistics,** Bayes’s theorem** (alternatively *Bayes’s law* or *Bayes’s rule*) describes the probability of an event, based on prior knowledge of conditions that might be related to the event.
Mathematically, it can be written as:

![formula.jpeg](attachment:formula.jpeg)

Where A and B are events and P(B)≠0
* P(A|B) is a conditional probability: the likelihood of event A occurring given that B is true.
* P(B|A) is also a conditional probability: the likelihood of event B occurring given that A is true.
* P(A) and P(B) are the probabilities of observing A and B respectively; they are known as the marginal probability.


Let’s understand it with the help of an example:

**The problem statement:**

You are planning a picnic today, but the morning is cloudy

Oh no! 50% of all rainy days start off cloudy!
But cloudy mornings are common (about 40% of days start cloudy)
And this is usually a dry month (only 3 of 30 days tend to be rainy, or 10%)
What is the chance of rain during the day?

We will use Rain to mean rain during the day, and Cloud to mean cloudy morning.

The chance of Rain given Cloud is written P(Rain|Cloud)

So let's put that in the formula:

$P(Rain|Cloud) = \frac{P(Rain)*P(Cloud|Rain)} {P(Cloud)}$          
                      


- P(Rain) is Probability of Rain = 10%
- P(Cloud|Rain) is Probability of Cloud, given that Rain happens = 50%
- P(Cloud) is Probability of Cloud = 40%

$P(Rain|Cloud) =  \frac{(0.1 x 0.5)} {0.4}   = .125$

Or a 12.5% chance of rain. Not too bad, let's have a picnic!

**Naïve:** It is called Naïve because it assumes that the occurrence of a certain feature is independent of the occurrence of other features. Such as if the fruit is identified on the bases of color, shape, and taste, then red, spherical, and sweet fruit is recognized as an apple. Hence each feature individually contributes to identify that it is an apple without depending on each other.<br>
**Bayes:** It is called Bayes because it depends on the principle of Bayes' Theorem

# Problem statement

Spam filtering using naive Bayes classifiers in order to predict whether a new mail based on its content, can be categorized as spam or not-spam.

### Data processing using panda library

In [1]:
# Import the required libraries

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import string
import matplotlib.pyplot as plt

In [2]:
# Load the dataset

data = pd.read_csv("spam.tsv",sep='\t',names=['Class','Message'])
data.head(8) # View the first 8 records of our dataset

,Class,Message
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!!
5,ham,As per your request 'Melle Melle (Oru Minnamin...
6,spam,WINNER!! As a valued network customer you have...
7,spam,Had your mobile 11 months or more? U R entitle...


In [3]:
# to view the first record
data.loc[0]

Class                                                    ham
Message    I've been searching for the right words to tha...
Name: 0, dtype: object

In [4]:
# Summary of the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5567 entries, 0 to 5566
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Class    5567 non-null   object
 1   Message  5567 non-null   object
dtypes: object(2)
memory usage: 87.1+ KB


In [5]:
# create a column to keep the count of the characters present in each record
data['Length'] = data['Message'].apply(len)

In [6]:
data["Length"].max()

910

In [7]:
data.loc[data["Length"] == data["Length"].min()]

,Class,Message,Length
1920,ham,Ok,2
3046,ham,Ok,2
4493,ham,Ok,2
5352,ham,Ok,2


In [8]:
# view the dataset with the column 'Length' which contains the number of characters present in each mail
data.head(10)

,Class,Message,Length
0,ham,I've been searching for the right words to tha...,196
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
2,ham,"Nah I don't think he goes to usf, he lives aro...",61
3,ham,Even my brother is not like to speak with me. ...,77
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!!,36
5,ham,As per your request 'Melle Melle (Oru Minnamin...,160
6,spam,WINNER!! As a valued network customer you have...,157
7,spam,Had your mobile 11 months or more? U R entitle...,154
8,ham,I'm gonna be home soon and i don't want to tal...,109
9,spam,"SIX chances to win CASH! From 100 to 20,000 po...",136


In [9]:
## The mails are categorised into 2 classes ie., spam and ham.
# Let's see the count of each class
data["Class"].value_counts()

Class
ham     4821
spam     746
Name: count, dtype: int64

### Data Visualization

In [10]:
data['Length'].describe() # to find the max length of the message.

count    5567.000000
mean       80.450153
std        59.891023
min         2.000000
25%        36.000000
50%        62.000000
75%       122.000000
max       910.000000
Name: Length, dtype: float64

See what we found, A 910 character long message. Let's use masking to find this message:

In [11]:
data['Length']==910

0       False
1       False
2       False
3       False
4       False
        ...  
5562    False
5563    False
5564    False
5565    False
5566    False
Name: Length, Length: 5567, dtype: bool

In [12]:
# the message that has the max characters
data[data['Length']==910]['Message']

1080    For me the love should start with attraction.i...
Name: Message, dtype: object

In [13]:
# view the message that has 910 characters in it
data[data['Length']==910]['Message'].iloc[0]

"For me the love should start with attraction.i should feel that I need her every time around me.she should be the first thing which comes in my thoughts.I would start the day and end it with her.she should be there every time I dream.love will be then when my every breath has her name.my life should happen around her.my life will be named to her.I would cry for her.will give all my happiness and take all her sorrows.I will be ready to fight with anyone for her.I will be in love when I will be doing the craziest things for her.love will be when I don't have to proove anyone that my girl is the most beautiful lady on the whole planet.I will always be singing praises for her.love will be when I start up making chicken curry and end up makiing sambar.life will be the most beautiful then.will get every morning and thank god for the day because she is with me.I would like to say a lot..will tell later.."

In [14]:
# View the message that has min characters
data[data['Length']==2]['Message'].iloc[0]

'Ok'

### Text Pre-Processing

In [15]:
data['Target']=data['Class'].map({"spam":0,'ham':1})

In [16]:
data.head()

,Class,Message,Length,Target
0,ham,I've been searching for the right words to tha...,196,1
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,0
2,ham,"Nah I don't think he goes to usf, he lives aro...",61,1
3,ham,Even my brother is not like to speak with me. ...,77,1
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!!,36,1


In [17]:
# Lets assign ham as 1
data.loc[data['Class']=="ham","Class"] = 1

In [18]:
# Lets assign spam as 0
data.loc[data['Class']=="spam","Class"] = 0

In [19]:
data.head(8)

,Class,Message,Length,Target
0,1,I've been searching for the right words to tha...,196,1
1,0,Free entry in 2 a wkly comp to win FA Cup fina...,155,0
2,1,"Nah I don't think he goes to usf, he lives aro...",61,1
3,1,Even my brother is not like to speak with me. ...,77,1
4,1,I HAVE A DATE ON SUNDAY WITH WILL!!!,36,1
5,1,As per your request 'Melle Melle (Oru Minnamin...,160,1
6,0,WINNER!! As a valued network customer you have...,157,0
7,0,Had your mobile 11 months or more? U R entitle...,154,0


First removing punctuation. We can just take advantage of Python's built-in string library to get a quick list of all the possible punctuation:

In [20]:
# the default list of punctuations
import string

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
# Why is it important to remove punctuation?

"This message is spam" == "This message is spam"

True

In [22]:
data.head()

,Class,Message,Length,Target
0,1,I've been searching for the right words to tha...,196,1
1,0,Free entry in 2 a wkly comp to win FA Cup fina...,155,0
2,1,"Nah I don't think he goes to usf, he lives aro...",61,1
3,1,Even my brother is not like to speak with me. ...,77,1
4,1,I HAVE A DATE ON SUNDAY WITH WILL!!!,36,1


In [23]:
# Let's remove the punctuation

def remove_punct(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text.lower()

data['text_clean'] = data['Message'].apply(remove_punct)

data.head()

,Class,Message,Length,Target,text_clean
0,1,I've been searching for the right words to tha...,196,1,ive been searching for the right words to than...
1,0,Free entry in 2 a wkly comp to win FA Cup fina...,155,0,free entry in 2 a wkly comp to win fa cup fina...
2,1,"Nah I don't think he goes to usf, he lives aro...",61,1,nah i dont think he goes to usf he lives aroun...
3,1,Even my brother is not like to speak with me. ...,77,1,even my brother is not like to speak with me t...
4,1,I HAVE A DATE ON SUNDAY WITH WILL!!!,36,1,i have a date on sunday with will


__Tokenization__ (process of converting the normal text strings in to a list of tokens(also known as lemmas)).

In [24]:
# original text and cleaned text
data.head(8)

,Class,Message,Length,Target,text_clean
0,1,I've been searching for the right words to tha...,196,1,ive been searching for the right words to than...
1,0,Free entry in 2 a wkly comp to win FA Cup fina...,155,0,free entry in 2 a wkly comp to win fa cup fina...
2,1,"Nah I don't think he goes to usf, he lives aro...",61,1,nah i dont think he goes to usf he lives aroun...
3,1,Even my brother is not like to speak with me. ...,77,1,even my brother is not like to speak with me t...
4,1,I HAVE A DATE ON SUNDAY WITH WILL!!!,36,1,i have a date on sunday with will
5,1,As per your request 'Melle Melle (Oru Minnamin...,160,1,as per your request melle melle oru minnaminun...
6,0,WINNER!! As a valued network customer you have...,157,0,winner as a valued network customer you have b...
7,0,Had your mobile 11 months or more? U R entitle...,154,0,had your mobile 11 months or more u r entitled...


Now we need to convert each of those messages into a vector the SciKit Learn's algorithm models can work with and machine learning model which we will gonig to use can understand.

In [25]:
# Countvectorizer is a method to convert text to numerical data.



[Stopwords are the words in any language which does not add much meaning to a sentence. They are the words which are very common in text documents such as a, an, the, you, your, etc. The Stop Words highly appear in text documents. However, they are not being helpful for text analysis in many of the cases, So it is better to remove from the text. We can focus on the important words if stop words have removed.]

In [26]:
# Splitting x and y

xSet = data['text_clean']
ySet = data['Class']


In [27]:
ySet

0       1
1       0
2       1
3       1
4       1
       ..
5562    0
5563    1
5564    1
5565    1
5566    1
Name: Class, Length: 5567, dtype: object

In [28]:
# Datatype for y is object. lets convert it into int
ySet = ySet.astype('int')
ySet

0       1
1       0
2       1
3       1
4       1
       ..
5562    0
5563    1
5564    1
5565    1
5566    1
Name: Class, Length: 5567, dtype: int32

In [29]:
xSet

0       ive been searching for the right words to than...
1       free entry in 2 a wkly comp to win fa cup fina...
2       nah i dont think he goes to usf he lives aroun...
3       even my brother is not like to speak with me t...
4                       i have a date on sunday with will
                              ...                        
5562    this is the 2nd time we have tried 2 contact u...
5563                  will ü b going to esplanade fr home
5564    pity  was in mood for that soany other suggest...
5565    the guy did some bitching but i acted like id ...
5566                            rofl its true to its name
Name: text_clean, Length: 5567, dtype: object

### Splitting Train and Test Data

In [30]:
xSet_train,xSet_test,ySet_train,ySet_test = train_test_split(xSet,ySet,test_size=0.2, random_state=10)

In [31]:
# Initialize the object for countvectorizer
CV = CountVectorizer(stop_words="english")
xSet_train_CV = CV.fit_transform(xSet_train)
xSet_train_CV

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 34532 stored elements and shape (4453, 8159)>

In [32]:
pd.DataFrame(xSet_train_CV.toarray())

,0,1,2,3,4,5,6,7,8,9,...,8149,8150,8151,8152,8153,8154,8155,8156,8157,8158
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4448,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4449,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4450,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4451,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Training a model

With messages represented as vectors, we can finally train our spam/ham classifier. Now we can actually use almost any sort of classification algorithms. For a variety of reasons, the Naive Bayes classifier algorithm is a good choice.

In [33]:
4453 * 8159

36332027

In [34]:
# Initialising the model
NB = MultinomialNB()

In [35]:
NB.fit(xSet_train_CV,ySet_train)

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [36]:
# Let's test CV on our test data
xSet_test_CV = CV.transform(xSet_test)

In [37]:
y_pred=NB.predict(xSet_test_CV)

In [38]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [39]:
# Checking accuracy

accuracyScore = accuracy_score(ySet_test,y_pred)*100

print("Prediction Accuracy :",accuracyScore)

Prediction Accuracy : 98.29443447037703


### SpamClassificationApplication

In [ ]:
msg = input("Enter Message: ") # to get the input message
msgInput = CV.transform([msg]) #
predict = NB.predict(msgInput)
if(predict[0]==0):
        print("------------------------MESSAGE-SENT-[CHECK-SPAM-FOLDER]---------------------------")
    else:
        print("---------------------------MESSAGE-SENT-[CHECK-INBOX]------------------------------")

## BAG OF WORDS

We cannot pass text directly to train our models in Natural Language Processing, thus we need to convert it into numbers, which machine can understand and can perform the required modelling on it

In [ ]:
# Let's understand it with a simple example

In [ ]:
# creating a list of sentences
documents = ["Dog bites man.", "Man bites dog.", "Dog eats meat.", "Man eats food."]

# Changing the text to lower case and remove the full stop from text
processed_docs = [doc.lower().replace(".","") for doc in documents]
processed_docs[3]

In [ ]:
# corpus is the collection of text
#look at the documents list
print("Our corpus: ", processed_docs)


    # Initialise the object for CountVectorizer
    count_vect = CountVectorizer()
            
    #Build a BOW representation for the corpus
    bow_rep = count_vect.fit_transform(processed_docs)
    
    #Look at the vocabulary mapping
    print("Our vocabulary: ", count_vect.vocabulary_)
    
    #see the BOW rep for first 2 document
    print("BoW representation for 'dog bites man': ", bow_rep[0].toarray())
    print("BoW representation for 'man bites dog: ",bow_rep[1].toarray())
    
    #Get the representation using this vocabulary, for a new text
    temp = count_vect.transform(["dog and dog are friends"])
    print("Bow representation for 'dog and dog are friends':", temp.toarray())

## TF-IDF

In **BOW approach** we saw so far, all the words in the text are treated equally important. There is no notion of some words in the document being more important than others. TF-IDF addresses this issue. It aims to quantify the importance of a given word relative to other words in the document and in the


<font color=darkviolet>  **Term Frequency (tf)** </font>
TF: Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization:

TF(t) = (Number of times term 't' appears in a document) / (Total number of terms in the document).



<font color=darkviolet>  **Inverse Document Frequency (idf)** </font>
              It measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following:

IDF(t) = log_e(Total number of documents / Number of documents with term t in it).corpus. It was commonly used representation scheme for information retrieval systems, for extracting relevant documents from a corpus for given text query.



__Let's see an example:__

Consider a document containing 100 words wherein the word cat appears 3 times.

The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03.

Now, assume we have 10 million documents and the word cat appears in one thousand of these.

Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4.

Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12

In [ ]:
# Splitting x and y

X = data['text_clean'].values
y = data['Class'].values
y

In [ ]:
# Datatype for y is object. lets convert it into int
y = y.astype('int')
y

In [ ]:
type(X)

In [ ]:
## text preprocessing and feature vectorizer
# To extract features from a document of words, we import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


tf=TfidfVectorizer() ## object creation
X=tf.fit_transform(X) ## fitting and transforming the data into vectors


In [ ]:
X.shape

In [ ]:
## print feature names selected from the raw documents
sorted(tf.vocabulary_)

In [ ]:
## number of features created
len(tf.get_feature_names())

In [ ]:
X

In [ ]:
## getting the feature vectors
X=X.toarray()

In [ ]:
## Creating training and testing
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=6)

In [ ]:
## Model creation
from sklearn.naive_bayes import BernoulliNB

## model object creation
nb=BernoulliNB(alpha=0.01)

## fitting the model
nb.fit(X_train,y_train)

## getting the prediction
y_hat=nb.predict(X_test)

In [ ]:
y_hat

In [ ]:
## Evaluating the model
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(y_test,y_hat))

In [ ]:
## confusion matrix
pd.crosstab(y_test,y_hat)

### Pros of Naive Bayes

- Naive Bayes Algorithm is a fast, highly scalable algorithm
- Naive Bayes can be classified for both binary classification and multi class classification. It provides different types of Naive Bayes Algorithms like GaussianNB, MultinominalNB, BernoulliNB.
- It is simple algorithm that depends on doing a bunch of count.
- Great choice for text classification problems. it's a popular choice for spam email classification.
- It can be easily trained on small datasets.
- Naive Bayes can handle misssing data, as they ignored when a probabilty is calculated for a class value.


### Cons of Naive Bayes

- It considers all the features to be unrelated, so it cannot learn the relationship between features. This limits the applicability of this algorithm in real-world use cases.
- Naive Bayes can learn individual featutre importance but can't determine the relationship among features.

## Application of Naive Bayes

##### Text classification / spam filtering / Sentiment analysis:
 - Naive Bayes classifiers mostly used in text classification
 - News article classification SPORTS, TECHNOLOGY etc.
 - Spam or Ham: Naive Bayes is the most popular method for mail filtering
 - Sentiment analysis focuses on identifying whether the customers think positively or negatively about a certain topic (product or service).


##### Recommendation System:
- Naive Bayes classifier and Collabrative filtering together buids a recommendation system that uses machine learning and data mining techniques to filter unseen information and predict whether a user would like a given resource or not.



### 3 Types of Naive Bayes in Scikit Learn

__Gaussian__

- It is used in classification and it assumes that features follow a normal distribution.

__Multinominal__
- It is used for discrete counts. For eg., let's say we have a text cLassification problem. Here we consider Bernoulli trails which is one step further and instead of "word occuring in the document", we have "count how often word occurs in the document" you can think of it as "number of times outcome number_x is observed over n trails".

__Bernoulli__
- The binomial model is useful if your feature vectors are binary (ie., Zeroes and One). One application would be text classification with 'bag of words' model where the 1s and 0s are "words occur in the document" and "word does not occur in the document" respectively.